# 07. Population

[1. 배경](#배경) <br>
[2. 데이터 읽고 인구 소멸 지역 계산](#데이터-읽기) <br>
&emsp;    - [컬럼 이름 변경 및 불필요한 행 제거](#컬럼-이름-변경)<br>
&emsp;    - [소멸 지역을 조사하기 위한 데이터 - 20-39세, 65세이상 컬럼 생성](#컬럼-생성)<br>
&emsp;    - [pivot_table](#pivot-table)<br>
&emsp;    - [소멸 비율 계산](#소멸-비율-계산)<br>
&emsp;    - [소멸 위기 지역 여부 컬럼 생성](#소멸-위기-지역-여부)<br>
&emsp;    - [서울시 모든 구의 유가 정보 엑셀 저장](#엑셀-저장2)<br>
[3. 셀프 주유소가 정말 저렴하나요? - 데이터 정리](#데이터-정리) <br>
&emsp;    - [수집한 서울시 모든 구의 유가 정보 엑셀을 한 번에 가져오기](#엑셀-불러오기)<br>
&emsp;    - [데이터프레임에 저장 및 정리](#데이터프레임-저장)<br>
[사용 함수 예제](#사용-함수-예제) <br>
&emsp;    - [fillna()](#fill-na)<br>

[Github에서는 toc가 작동안함. nbviewer로 보기👀](https://nbviewer.org/github/solpinetree/ds_study/blob/main/source_code/04.%20Self%20Oil%20Station%20Price%20Analysis.ipynb)

<a id="배경"></a>
## 1. 배경
목표 
- 1. 인구 소멸 위기 지역 파악
- 2. 인구 소멸 위기 지역의 위도 표현
- 3. 지도 표현에 대한 카르토그램 표현

---

<a id="데이터-읽기"></a>
## 2. 데이터 읽고 인구 소멸 지역 계산

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import set_matplotlib_hangul
import warnings

warnings.filterwarnings(action="ignore") # 경고 문구 무시
%matplotlib inline

In [66]:
population = pd.read_csv(
    "../data/07. population_raw_data.csv", 
    encoding="utf-8",
    header=1
)
population.fillna(method="pad", inplace=True)
population.head()

,행정구역(동읍면)별,시도,항목,계,20 - 24세,25 - 29세,30 - 34세,35 - 39세,40 - 44세,45 - 49세,...,55 - 59세,60 - 64세,65 - 69세,70 - 74세,75 - 79세,80 - 84세,85 - 89세,90 - 94세,95 - 99세,100+
0,전국,소계,총인구수 (명),51265238,2711038,3343400,3454825,3144285,3968671,3829092,...,4260354,4216718,3526690,2332941,1751817,1340443,746668,259885,57336,8688
1,전국,소계,남자인구수 (명),25528507,1408133,1748364,1817952,1622003,2021792,1942815,...,2138774,2096981,1716169,1096071,783469,526625,242361,62946,10567,1568
2,전국,소계,여자인구수 (명),25736731,1302905,1595036,1636873,1522282,1946879,1886277,...,2121580,2119737,1810521,1236870,968348,813818,504307,196939,46769,7120
3,서울특별시,소계,총인구수 (명),9360400,543819,788133,790442,644805,722803,674392,...,709798,700953,622332,427050,330610,233241,116634,38549,9209,1505
4,서울특별시,소계,남자인구수 (명),4521316,253919,374777,396396,321523,358815,331957,...,347271,336023,290356,195609,146867,99161,42729,10796,2072,351


In [67]:
population.columns

Index(['행정구역(동읍면)별', '시도', '항목', '계', '20 - 24세', '25 - 29세', '30 - 34세',
       '35 - 39세', '40 - 44세', '45 - 49세', '50 - 54세', '55 - 59세', '60 - 64세',
       '65 - 69세', '70 - 74세', '75 - 79세', '80 - 84세', '85 - 89세', '90 - 94세',
       '95 - 99세', '100+'],
      dtype='object')

<a id="컬럼-이름-변경"></a>
### 컬럼 이름 변경 및 불필요한 행 제거

In [68]:
population.rename(
    columns={"행정구역(동읍면)별": "광역시도", "계" : "인구수"},
    inplace=True
)
population.head()

,광역시도,시도,항목,인구수,20 - 24세,25 - 29세,30 - 34세,35 - 39세,40 - 44세,45 - 49세,...,55 - 59세,60 - 64세,65 - 69세,70 - 74세,75 - 79세,80 - 84세,85 - 89세,90 - 94세,95 - 99세,100+
0,전국,소계,총인구수 (명),51265238,2711038,3343400,3454825,3144285,3968671,3829092,...,4260354,4216718,3526690,2332941,1751817,1340443,746668,259885,57336,8688
1,전국,소계,남자인구수 (명),25528507,1408133,1748364,1817952,1622003,2021792,1942815,...,2138774,2096981,1716169,1096071,783469,526625,242361,62946,10567,1568
2,전국,소계,여자인구수 (명),25736731,1302905,1595036,1636873,1522282,1946879,1886277,...,2121580,2119737,1810521,1236870,968348,813818,504307,196939,46769,7120
3,서울특별시,소계,총인구수 (명),9360400,543819,788133,790442,644805,722803,674392,...,709798,700953,622332,427050,330610,233241,116634,38549,9209,1505
4,서울특별시,소계,남자인구수 (명),4521316,253919,374777,396396,321523,358815,331957,...,347271,336023,290356,195609,146867,99161,42729,10796,2072,351


In [69]:
population = population[(population["시도"] != "소계")]
population.head()

,광역시도,시도,항목,인구수,20 - 24세,25 - 29세,30 - 34세,35 - 39세,40 - 44세,45 - 49세,...,55 - 59세,60 - 64세,65 - 69세,70 - 74세,75 - 79세,80 - 84세,85 - 89세,90 - 94세,95 - 99세,100+
6,서울특별시,종로구,총인구수 (명),139093,8985,12386,11225,8374,9635,9136,...,11482,11361,9599,6329,5380,4346,2408,825,220,44
7,서울특별시,종로구,남자인구수 (명),66956,4056,5979,5656,4073,4721,4391,...,5628,5738,4759,3005,2348,1788,913,279,57,14
8,서울특별시,종로구,여자인구수 (명),72137,4929,6407,5569,4301,4914,4745,...,5854,5623,4840,3324,3032,2558,1495,546,163,30
9,서울특별시,중구,총인구수 (명),121407,6534,10613,11921,9464,9071,7585,...,9458,9641,8793,5888,4790,3765,1950,724,188,24
10,서울특별시,중구,남자인구수 (명),58546,2930,4797,5886,4749,4579,3895,...,4800,4858,4300,2793,2060,1524,644,185,46,8


In [70]:
population.rename(columns={"항목":"구분"}, inplace=True)

population.loc[population["구분"] == "총인구수 (명)", "구분"] = "합계"
population.loc[population["구분"] == "남자인구수 (명)", "구분"] = "남자"
population.loc[population["구분"] == "여자인구수 (명)", "구분"] = "여자"

population.head()

,광역시도,시도,구분,인구수,20 - 24세,25 - 29세,30 - 34세,35 - 39세,40 - 44세,45 - 49세,...,55 - 59세,60 - 64세,65 - 69세,70 - 74세,75 - 79세,80 - 84세,85 - 89세,90 - 94세,95 - 99세,100+
6,서울특별시,종로구,합계,139093,8985,12386,11225,8374,9635,9136,...,11482,11361,9599,6329,5380,4346,2408,825,220,44
7,서울특별시,종로구,남자,66956,4056,5979,5656,4073,4721,4391,...,5628,5738,4759,3005,2348,1788,913,279,57,14
8,서울특별시,종로구,여자,72137,4929,6407,5569,4301,4914,4745,...,5854,5623,4840,3324,3032,2558,1495,546,163,30
9,서울특별시,중구,합계,121407,6534,10613,11921,9464,9071,7585,...,9458,9641,8793,5888,4790,3765,1950,724,188,24
10,서울특별시,중구,남자,58546,2930,4797,5886,4749,4579,3895,...,4800,4858,4300,2793,2060,1524,644,185,46,8


<a id="컬럼-생성"></a>
### 소멸 지역을 조사하기 위한 데이터 - 20-39세, 65세이상 컬럼 생성

In [71]:
population["20-39세"] = (
    population["20 - 24세"] 
    + population["25 - 29세"]
    + population["30 - 34세"]
    + population["35 - 39세"]
)

population["65세이상"] = (
    population["65 - 69세"]
    + population["70 - 74세"]
    + population["75 - 79세"]
    + population["80 - 84세"]
    + population["85 - 89세"]
    + population["90 - 94세"]
    + population["95 - 99세"]
    + population["100+"]
)

population.head()

,광역시도,시도,구분,인구수,20 - 24세,25 - 29세,30 - 34세,35 - 39세,40 - 44세,45 - 49세,...,65 - 69세,70 - 74세,75 - 79세,80 - 84세,85 - 89세,90 - 94세,95 - 99세,100+,20-39세,65세이상
6,서울특별시,종로구,합계,139093,8985,12386,11225,8374,9635,9136,...,9599,6329,5380,4346,2408,825,220,44,40970,29151
7,서울특별시,종로구,남자,66956,4056,5979,5656,4073,4721,4391,...,4759,3005,2348,1788,913,279,57,14,19764,13163
8,서울특별시,종로구,여자,72137,4929,6407,5569,4301,4914,4745,...,4840,3324,3032,2558,1495,546,163,30,21206,15988
9,서울특별시,중구,합계,121407,6534,10613,11921,9464,9071,7585,...,8793,5888,4790,3765,1950,724,188,24,38532,26122
10,서울특별시,중구,남자,58546,2930,4797,5886,4749,4579,3895,...,4300,2793,2060,1524,644,185,46,8,18362,11560


<a id="pivot-table"></a>
### pivot_table

In [72]:
pop = pd.pivot_table(
    population,
    index=["광역시도", "시도"], 
    columns=["구분"],
    values=["인구수", "20-39세", "65세이상"]
)

pop

20-39세                  65세이상                    인구수          \
구분                 남자      여자      합계     남자     여자      합계      남자      여자   
광역시도    시도                                                                    
강원특별자치도 강릉시     22901   19250   42151  23028  30168   53196  103188  105063   
        고성군      2823    1956    4779   3934   5028    8962   13909   13234   
        동해시      9335    7056   16391   9407  12016   21423   44555   43297   
        동해출장소       0       0       0      0      0       0       0       0   
        삼척시      6352    4991   11343   8041  10378   18419   31726   30604   
...               ...     ...     ...    ...    ...     ...     ...     ...   
충청북도    제천시     13861   11241   25102  15614  18977   34591   65156   64229   
        증평군      4866    3686    8552   3521   4394    7915   19395   17840   
        진천군     11074    8290   19364   7397   9008   16405   45907   40862   
        청주시    123802  104519  228321  61307  77676  138983  431274  422289   
        충주시     24764   18931   43695  21946  27471   49417  105235  102111   

                       
구분                 합계  
광역시도    시도             
강원특별자치도 강릉시    208251  
        고성군     27143  
        동해시     87852  
        동해출장소       0  
        삼척시     62330  
...               ...  
충청북도    제천시    129385  
        증평군     37235  
        진천군     86769  
        청주시    853563  
        충주시    207346  

[230 rows x 9 columns]

<a id="소멸-비율-계산"></a>
### 소멸 비율 계산
- 젊은 여성 인구가 노인 인구의 절반에 미달할 경우 '소멸 위험 지역'으로 분류하는 방식 에서 착안한 계산법

In [73]:
pop["소멸비율"] = pop["20-39세", "여자"] / (pop["65세이상", "합계"] / 2)
pop.head()

20-39세                65세이상                   인구수          \
구분                남자     여자     합계     남자     여자     합계      남자      여자   
광역시도    시도                                                                
강원특별자치도 강릉시    22901  19250  42151  23028  30168  53196  103188  105063   
        고성군     2823   1956   4779   3934   5028   8962   13909   13234   
        동해시     9335   7056  16391   9407  12016  21423   44555   43297   
        동해출장소      0      0      0      0      0      0       0       0   
        삼척시     6352   4991  11343   8041  10378  18419   31726   30604   

                           소멸비율  
구분                 합계            
광역시도    시도                       
강원특별자치도 강릉시    208251  0.723739  
        고성군     27143  0.436510  
        동해시     87852  0.658731  
        동해출장소       0       NaN  
        삼척시     62330  0.541940

In [74]:
# 동해출장소 데이터 삭제
pop.drop([('강원특별자치도', '동해출장소')], axis=0, inplace=True)
pop.head()

20-39세                65세이상                   인구수                  \
구분              남자     여자     합계     남자     여자     합계      남자      여자      합계   
광역시도    시도                                                                      
강원특별자치도 강릉시  22901  19250  42151  23028  30168  53196  103188  105063  208251   
        고성군   2823   1956   4779   3934   5028   8962   13909   13234   27143   
        동해시   9335   7056  16391   9407  12016  21423   44555   43297   87852   
        삼척시   6352   4991  11343   8041  10378  18419   31726   30604   62330   
        속초시   9224   7344  16568   8294  10757  19051   40216   41208   81424   

                 소멸비율  
구분                     
광역시도    시도             
강원특별자치도 강릉시  0.723739  
        고성군  0.436510  
        동해시  0.658731  
        삼척시  0.541940  
        속초시  0.770983

<a id="소멸-위기-지역-여부"></a>
### 소멸 위기 지역 여부 컬럼 생성

In [75]:
pop["소멸위기지역"] = pop["소멸비율"] < 1.0
pop.head()

20-39세                65세이상                   인구수                  \
구분              남자     여자     합계     남자     여자     합계      남자      여자      합계   
광역시도    시도                                                                      
강원특별자치도 강릉시  22901  19250  42151  23028  30168  53196  103188  105063  208251   
        고성군   2823   1956   4779   3934   5028   8962   13909   13234   27143   
        동해시   9335   7056  16391   9407  12016  21423   44555   43297   87852   
        삼척시   6352   4991  11343   8041  10378  18419   31726   30604   62330   
        속초시   9224   7344  16568   8294  10757  19051   40216   41208   81424   

                 소멸비율 소멸위기지역  
구분                            
광역시도    시도                    
강원특별자치도 강릉시  0.723739   True  
        고성군  0.436510   True  
        동해시  0.658731   True  
        삼척시  0.541940   True  
        속초시  0.770983   True

In [76]:
len(pop)

229

In [77]:
# 소멸위기지역 조회

pop[pop["소멸위기지역"]==True].index.get_level_values(1)

Index(['강릉시', '고성군', '동해시', '삼척시', '속초시', '양구군', '양양군', '영월군', '인제군', '정선군',
       ...
       '홍성군', '괴산군', '단양군', '보은군', '영동군', '옥천군', '음성군', '제천시', '증평군', '충주시'],
      dtype='object', name='시도', length=132)

In [78]:
pop.reset_index(inplace=True)
pop.head()

광역시도   시도 20-39세                65세이상                   인구수          \
구분                   남자     여자     합계     남자     여자     합계      남자      여자   
0   강원특별자치도  강릉시  22901  19250  42151  23028  30168  53196  103188  105063   
1   강원특별자치도  고성군   2823   1956   4779   3934   5028   8962   13909   13234   
2   강원특별자치도  동해시   9335   7056  16391   9407  12016  21423   44555   43297   
3   강원특별자치도  삼척시   6352   4991  11343   8041  10378  18419   31726   30604   
4   강원특별자치도  속초시   9224   7344  16568   8294  10757  19051   40216   41208   

                소멸비율 소멸위기지역  
구분      합계                   
0   208251  0.723739   True  
1    27143  0.436510   True  
2    87852  0.658731   True  
3    62330  0.541940   True  
4    81424  0.770983   True

In [79]:
pop.columns.get_level_values(0)

Index(['광역시도', '시도', '20-39세', '20-39세', '20-39세', '65세이상', '65세이상', '65세이상',
       '인구수', '인구수', '인구수', '소멸비율', '소멸위기지역'],
      dtype='object')

In [80]:
tmp_columns=[
    pop.columns.get_level_values(0)[n] + pop.columns.get_level_values(1)[n]
    for n in range(0, len(pop.columns.get_level_values(0)))
]
pop.columns = tmp_columns
pop.head()

,광역시도,시도,20-39세남자,20-39세여자,20-39세합계,65세이상남자,65세이상여자,65세이상합계,인구수남자,인구수여자,인구수합계,소멸비율,소멸위기지역
0,강원특별자치도,강릉시,22901,19250,42151,23028,30168,53196,103188,105063,208251,0.723739,True
1,강원특별자치도,고성군,2823,1956,4779,3934,5028,8962,13909,13234,27143,0.436510,True
2,강원특별자치도,동해시,9335,7056,16391,9407,12016,21423,44555,43297,87852,0.658731,True
3,강원특별자치도,삼척시,6352,4991,11343,8041,10378,18419,31726,30604,62330,0.541940,True
4,강원특별자치도,속초시,9224,7344,16568,8294,10757,19051,40216,41208,81424,0.770983,True


---

<a id="사용-함수-예제"></a>
## 사용 함수 예제

<a id="fill-na"></a>
### fillna()

In [3]:
datas = {
    "A": np.random.randint(1, 45, 8),
    "B": np.random.randint(1, 45, 8),
    "C": np.random.randint(1, 45, 8),
}

datas

{'A': array([34, 32,  2, 32, 32,  2, 42, 34]),
 'B': array([20, 29, 24,  2,  5, 23, 30, 44]),
 'C': array([43, 13, 10, 29, 10, 33,  2, 11])}

In [4]:
fillna_df = pd.DataFrame(datas)
fillna_df

,A,B,C
0,34,20,43
1,32,29,13
2,2,24,10
3,32,2,29
4,32,5,10
5,2,23,33
6,42,30,2
7,34,44,11


In [6]:
fillna_df.loc[2:4, ["A"]] = np.nan
fillna_df.loc[3:5, ["B"]] = np.nan
fillna_df.loc[4:7, ["C"]] = np.nan
fillna_df

,A,B,C
0,34.0,20.0,43.0
1,32.0,29.0,13.0
2,NaN,24.0,10.0
3,NaN,NaN,29.0
4,NaN,NaN,NaN
5,2.0,NaN,NaN
6,42.0,30.0,NaN
7,34.0,44.0,NaN


####  method
```
method : {'backfill', 'bfill', 'pad', 'ffill', None}, default None
    Method to use for filling holes in reindexed Series
    pad / ffill: propagate last valid observation forward to next valid
    backfill / bfill: use next valid observation to fill gap.
```

In [7]:
fillna_df.fillna(method="pad")

,A,B,C
0,34.0,20.0,43.0
1,32.0,29.0,13.0
2,32.0,24.0,10.0
3,32.0,24.0,29.0
4,32.0,24.0,29.0
5,2.0,24.0,29.0
6,42.0,30.0,29.0
7,34.0,44.0,29.0


In [8]:
fillna_df.fillna(method="backfill")

,A,B,C
0,34.0,20.0,43.0
1,32.0,29.0,13.0
2,2.0,24.0,10.0
3,2.0,30.0,29.0
4,2.0,30.0,NaN
5,2.0,30.0,NaN
6,42.0,30.0,NaN
7,34.0,44.0,NaN


In [9]:
fillna_df.fillna(method="backfill", axis=1)

,A,B,C
0,34.0,20.0,43.0
1,32.0,29.0,13.0
2,24.0,24.0,10.0
3,29.0,29.0,29.0
4,NaN,NaN,NaN
5,2.0,NaN,NaN
6,42.0,30.0,NaN
7,34.0,44.0,NaN
